# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Save the directory
directory = '/content/drive/MyDrive/GenAI/OpenAI/OpenAI Project/Dim Sum'

In [ ]:
!pip install openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('genai_course')

In [ ]:
# Load the libraries
from openai import OpenAI
import os
import base64
from IPython.display import Image, display, Markdown
import pandas as pd

In [ ]:
# Set the OpenAI connection and model
MODEL = "gpt-4o"
client = OpenAI(api_key=openai_api_key)

In [ ]:
system_prompt = """
Convert the menu image to a structured excel sheet format following the provided template and instructions.
This assistant converts restaurant or cafe menu data into a structured Excel sheet that adheres to a specific template.
The template includes categories, subscategories, item names, prices, descriptions, and more, ensuring data consistency.
This assistant helps user fill out each row correctly, dollwoing the detailed instructions provided.

Overview:
- Each row in the Excel spreadsheet represents a unique item, categorized under a category or subcategory.
- Category and subcategory names are repeated for items within the same subcategory.
- Certain columns are left blank when not applicable, such as subcategory details for items directly under a category.
- Item details, including names, prices, and descriptions, must be unique for each entry.
- Uploaded menu content will be appended to the existing menu without deleting any current entries.

Columns Guide:

Column Name                    | Description                               | Accepted Values           | Example
-------------------------------|-------------------------------------------|---------------------------|-----------------------
CategoryTitlePt (Column A)      | Category names in Portuguese              | Text, 256 characters max  | Bebidas
CategoryTitleEn (Column B) (Optional) | English translations of category titles | Text, 256 characters max  | Beverages
SubcategoryTitlePt (Column C) (Optional) | Subcategory titles in Portuguese | Text, 256 characters max or blank | Sucos
SubcategoryTitleEn (Column D) (Optional) | English translations of subcategory titles | Text, 256 characters max or blank | Juices
ItemNamePt (Column E)           | Item names in Portuguese                  | Text, 256 characters max  | Água Mineral
ItemNameEn (Column F) (Optional) | English translations of item names | Text, 256 characters max or blank | Mineral Water
ItemPrice (Column G)          | Price of each item without currency symbol  | Text                      | 2.50 or 2,50
Calories (Column H) (Optional) | Caloric content of each item              | Numeric                   | 150
PortionSize (Column I)        | Portion size for each item in units        | Text                      | 500ml, 1, 2-3
Availability (Column J) (Optional) | Current availability of the item     | Numeric: 1 for Yes, 0 for No | 1
ItemDescriptionPt (Column K) (Optional) | Detailed description in Portuguese | Text, 500 characters max  | Contains essential minerals
ItemDescriptionEn (Column L) (Optional) | Detailed description in English | Text, 500 characters max  | Contains essential minerals

Notes:
- Ensure all data entered follows the specified formats to maintain database integrity.
- Review the data for accuracy and consistency before submitting the Excel sheet.
"""


In [ ]:
# Get the directory
os.chdir(directory)
IMAGE_DIR = directory

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Process imaged in the directory
image_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
image_files

['DimSum Amoreiras 1.PNG',
 'DimSum Amoreiras 2.PNG',
 'DimSum Amoreiras 3.PNG',
 'DimSum Amoreiras 4.PNG',
 'DimSum Amoreiras 5.PNG']

In [ ]:
# Prompt the user for the excel file name
new_excel_file_name = input("Enter the new Excel file name (without extension): ")
EXCEL_PATH = os.path.join(directory, f"{new_excel_file_name}.xlsx")

# Create the PANDAS dataframe
df = pd.DataFrame(columns=['CategoryTitlePt', 'CategoryTitleEn', 'SubcategoryTitlePt', 'SubcategoryTitleEn',
                           'ItemNamePt', 'ItemNameEn', 'ItemPrice', 'Calories', 'PortionSize', 'Availability',
                           'ItemDescriptionPt', 'ItemDescriptionEn'])

for image in image_files:
  # Retrieve and encode the image
  image_path = os.path.join(IMAGE_DIR, image)
  image_data = encode_image(image_path)

  # Adding a flag for the headers
  headers_added = False

  # Use GPT-4o to analyze and convert the imae
  response = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": [
              {'type': 'text',
              'text': "Convert this menu image to a structured Excel Sheet Format."},
              {'type': 'image_url',
              'image_url': {'url': f'data:image/png;base64,{image_data}'}}
          ]}],
      temperature = 0
  )

  for row in response.choices[0].message.content.split('\n'):
    if row.startswith('|') and not row.startswith('|-'): # Ensure that the data is a row and not a header format
      columns = [col.strip() for col in row.split('|')[1:-1]]
      if len(columns) == len(df.columns):
        if 'CategoryTitlePt' in columns:
          headers_added = True
          continue
        if headers_added and 'CategoryTitlePt' in columns:
          continue # skip the row
        new_row = pd.Series(columns, index=df.columns)
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
      else:
        print(f"Skipping row { row}")

df.to_excel(EXCEL_PATH, index=False)
print(f"Excel file saved at: {EXCEL_PATH}")

In [ ]:
df

,CategoryTitlePt,CategoryTitleEn,SubcategoryTitlePt,SubcategoryTitleEn,ItemNamePt,ItemNameEn,ItemPrice,Calories,PortionSize,Availability,ItemDescriptionPt,ItemDescriptionEn
0,Sopas,Soups,,,Sopa Won Ton,Won Ton Soup,"3,95",,1 unid,1,"Paksoy, Porco e Camarão","Paksoy, Pork & Shrimp"
1,Sopas,Soups,,,Sopa Vegetariana,Vegetarian Soup,"3,95",,1 unid,1,"Paksoy, Bambu, Cogumelo e Verduras","Paksoy, Bamboo, Mushroom & Vegetables"
2,Arroz & Verduras,Rice & Vegetables,,,Arroz branco,Steamed Rice,"1,95",,1 unid,1,Arroz ao vapor,Steamed Rice
3,Arroz & Verduras,Rice & Vegetables,,,Paksoy ao vapor,Steamed Paksoy,"2,40",,1 unid,1,Paksoy ao vapor com Coentros,Steamed Paksoy with Coriander
4,Siao Long Pao,Siao Long Pao,,,Siao Long Pao Tradicional,Traditional Siao Long Pao,"4,50",,3 unid,1,"Porco, Paksoy e Cogumelo Shiitake","Pork, Paksoy & Shiitake Mushroom"
5,Siao Long Pao,Siao Long Pao,,,Siao Long Pao Negro,Black Siao Long Pao,"4,50",,3 unid,1,"Porco, Gengibre e Cebolinho","Pork, Ginger & Chives"
6,Gyozas,Gyozas,,,Gyoza Vegetais,Vegetable Gyoza,"4,50",,3 unid,1,Veg,Veg
7,Gyozas,Gyozas,,,Gyoza Camarão,Shrimp Gyoza,"4,50",,3 unid,1,Shrimp,Shrimp
8,Gyozas,Gyozas,,,Gyoza Pato,Duck Gyoza,"4,50",,3 unid,1,Duck,Duck
9,Gyozas,Gyozas,,,Gyoza Galinha,Chicken Gyoza,"4,50",,3 unid,1,Chicken,Chicken
